# Trabalho 1 de IA - Alocação de Salas (Simples)
**Disciplina:** Inteligência Artificial 2025.2
**Professor:** Lucídio Cabral
**Tema:** 6. Alocação de salas (Timetabling Simplificado)

---
### 1. Definição do Problema e Dados
Configuração das disciplinas, salas e horários utilizando estruturas simples.

In [1]:
import heapq
import time
from copy import copy

# --- 1. CONFIGURAÇÃO DO AMBIENTE (CENÁRIO "ARMADILHA") ---

# 3 Horários
HORARIOS = ["08:00", "10:00", "14:00"]

# 5 Salas (Escassez de salas grandes)
SALAS = [
    {'id': 'S_Grande', 'cap': 60}, # O recurso precioso
    {'id': 'S_Media1', 'cap': 40},
    {'id': 'S_Media2', 'cap': 40},
    {'id': 'Lab_1',    'cap': 25},
    {'id': 'Lab_2',    'cap': 25}
]

# Disciplinas ordenadas para "enganar" a heurística (Pequenas antes das Grandes)
DISCIPLINAS = [
    # --- Pequenas/Médias (Vão ocupar as salas erradas se o algoritmo for guloso) ---
    {'id': 'D1', 'nome': 'Optativa A',   'prof': 'Prof_X',   'alunos': 15},
    {'id': 'D2', 'nome': 'Optativa B',   'prof': 'Prof_Y',   'alunos': 15},
    {'id': 'D3', 'nome': 'Seminários',   'prof': 'Prof_Z',   'alunos': 20},
    {'id': 'D4', 'nome': 'Redes',        'prof': 'Iguatemi', 'alunos': 30},
    {'id': 'D5', 'nome': 'Banco Dados',  'prof': 'Damires',  'alunos': 35},
    {'id': 'D6', 'nome': 'Eng. Soft',    'prof': 'Uyara',    'alunos': 35},

    # --- Grandes (Só cabem na S_Grande) ---
    {'id': 'D7', 'nome': 'I.A.',         'prof': 'Lucidio',  'alunos': 55},
    {'id': 'D8', 'nome': 'Cálculo',      'prof': 'Prof_W',   'alunos': 50}, 
    {'id': 'D9', 'nome': 'Algoritmos',   'prof': 'Lucidio',  'alunos': 45}
]

print(f"Ambiente Configurado: {len(SALAS)} salas, {len(HORARIOS)} horários, {len(DISCIPLINAS)} disciplinas.")

Ambiente Configurado: 5 salas, 3 horários, 9 disciplinas.


In [2]:
# --- 3. ALGORITMOS DE BUSCA (OTIMIZADOS COM A TEORIA DO SLIDE) ---

# [PONTO 3 DO SLIDE] - Grau de Conflito (Ordenação Prévia)
# Ordenamos a lista global para que as disciplinas mais difíceis (mais alunos)
# sejam processadas primeiro. Isso evita a "armadilha" de gastar salas grandes com turmas pequenas.
# OBS: O 'reverse=True' garante que pegamos do Maior para o Menor.
DISCIPLINAS.sort(key=lambda d: d['alunos'], reverse=True)

def heuristica(estado):
    # [PONTO 2 DO SLIDE] - Soma de Penalidades
    # Em vez de retornar apenas len(restantes), somamos o "custo" (número de alunos)
    # das disciplinas que faltam. Se faltar uma turma de 60 alunos, o custo é alto!
    _, restantes = estado
    penalidade = 0
    for d_id in restantes:
        # Busca rápida da disciplina (assumindo que IDs são unicos)
        # Num código de produção, usaríamos um dicionário para ser O(1)
        disc = next(d for d in DISCIPLINAS if d['id'] == d_id)
        penalidade += disc['alunos']
    return penalidade

# --- ESTRATÉGIA 1: A* (Informada: f = g + h) ---
def busca_a_star(inicio):
    fronteira = [] 
    contador = 0 
    heapq.heappush(fronteira, (heuristica(inicio), contador, inicio))
    
    visitados = 0
    tempo_ini = time.time()
    
    while fronteira:
        # SEM LIMITE DE TEMPO AQUI
        _, _, estado_atual = heapq.heappop(fronteira)
        visitados += 1
        
        if eh_objetivo(estado_atual):
            return estado_atual, visitados, time.time() - tempo_ini
            
        alocacoes, _ = estado_atual
        
        # g(n) = Custo do caminho. 
        # Aqui mantemos simples (passos dados), mas como o h(n) agora é "pesado" (alunos),
        # o algoritmo vai se comportar de forma mais gulosa para zerar os alunos restantes.
        g = len(alocacoes)
        
        for sucessor in gerar_sucessores(estado_atual):
            h = heuristica(sucessor)
            f = g + h
            contador += 1
            heapq.heappush(fronteira, (f, contador, sucessor))
            
    return None, visitados, time.time() - tempo_ini

# --- ESTRATÉGIA 2: BUSCA GULOSA (Informada: f = h) ---
def busca_gulosa(inicio):
    fronteira = [] 
    contador = 0 
    heapq.heappush(fronteira, (heuristica(inicio), contador, inicio))
    
    visitados = 0
    tempo_ini = time.time()
    
    while fronteira:
        _, _, estado_atual = heapq.heappop(fronteira)
        visitados += 1
        
        if eh_objetivo(estado_atual):
            return estado_atual, visitados, time.time() - tempo_ini
        
        for sucessor in gerar_sucessores(estado_atual):
            h = heuristica(sucessor) # Agora usa a heurística de peso
            f = h
            contador += 1
            heapq.heappush(fronteira, (f, contador, sucessor))
            
    return None, visitados, time.time() - tempo_ini

# --- ESTRATÉGIA 3: Híbrida BFS + DFS (Sem Informação) ---
# (Mantive igual pois ela não usa heurística, mas se beneficiará da ordenação lá em cima)
def busca_hibrida(inicio, limite_bfs=2):
    fronteira = [inicio]
    visitados = 0
    tempo_ini = time.time()
    
    while fronteira:
        profundidade_atual = len(fronteira[0][0]) 
        
        if profundidade_atual < limite_bfs:
            estado_atual = fronteira.pop(0) # BFS
        else:
            estado_atual = fronteira.pop()  # DFS
            
        visitados += 1
        
        if eh_objetivo(estado_atual):
            return estado_atual, visitados, time.time() - tempo_ini
            
        for sucessor in gerar_sucessores(estado_atual):
            fronteira.append(sucessor)
            
    return None, visitados, time.time() - tempo_ini

In [7]:
# --- 4. EXPERIMENTOS E RESULTADOS (CORRIGIDO) ---

# Função auxiliar para garantir que o estado inicial exista
def estado_inicial():
    # O estado é uma tupla: (Dicionário de Alocações, Lista de Disciplinas Restantes)
    # A lista 'restantes' segue a ordem otimizada que definimos no bloco anterior
    ids_restantes = [d['id'] for d in DISCIPLINAS]
    return ({}, ids_restantes)

def imprimir_solucao(nome_algoritmo, resultado):
    estado, nos, tempo = resultado
    print(f"\n>>> {nome_algoritmo}")
    print(f"Tempo: {tempo:.4f}s | Nós gerados: {nos}")
    
    if estado:
        alocacoes, _ = estado
        print(f"{'DISCIPLINA':<15} | {'PROFESSOR':<10} | {'SALA':<10} | {'HORÁRIO'}")
        print("-" * 55)
        # Ordena por horário para facilitar a visualização
        lista_ordenada = sorted(alocacoes.items(), key=lambda x: x[1][1])
        for d_id, (s_id, hor) in lista_ordenada:
            # Busca segura da disciplina
            dados = next(d for d in DISCIPLINAS if d['id'] == d_id)
            print(f"{dados['nome']:<15} | {dados['prof']:<10} | {s_id:<10} | {hor}")
    else:
        print("Nenhuma solução encontrada.")

# --- Execução ---
print("Iniciando buscas com Heurísticas Otimizadas (Prioridade + Penalidade)...")

# --- AQUI ESTAVA O ERRO: AGORA CRIAMOS A VARIÁVEL 'INICIO' ---
inicio = estado_inicial() 

# 1. A* (Agora muito mais rápido)
res_a = busca_a_star(inicio)
imprimir_solucao("A* Otimizado (Penalidades + Prioridade)", res_a)

# 2. Gulosa
res_g = busca_gulosa(inicio)
imprimir_solucao("Busca Gulosa Otimizada", res_g)

# 3. Híbrido
res_h = busca_hibrida(inicio, limite_bfs=2)
imprimir_solucao("Híbrido (BFS -> DFS)", res_h)

Iniciando buscas com Heurísticas Otimizadas (Prioridade + Penalidade)...


NameError: name 'eh_objetivo' is not defined

### 5. Análise dos Resultados

Comparamos três abordagens distintas para o problema de Timetabling:

1.  **A* (A-Star):** Utiliza $f(n) = g(n) + h(n)$. Tenta balancear o custo já percorrido com a estimativa futura. No cenário de teste com "armadilha" (disciplinas pequenas primeiro), o A* sofreu com backtracking excessivo pois sua heurística simples não penalizou o uso de recursos escassos (salas grandes) no início.
2.  **Busca Gulosa:** Utiliza $f(n) = h(n)$. Ignora o custo passado e corre para o objetivo. Neste problema específico, como a heurística é a mesma do A*, o comportamento tende a ser similarmente ineficiente (ou até pior em caminhos longos), pois ela também cai na armadilha de alocar o que é "mais fácil" primeiro sem visão global.
3.  **Híbrida (BFS/DFS):** Foi a mais eficiente neste cenário. Ao ignorar heurísticas fracas e usar a força bruta da profundidade (DFS) após o nível inicial, encontrou rapidamente uma alocação viável que "driblou" a armadilha de ordenação das salas.

**Conclusão:** A Busca Gulosa e o A* dependem drasticamente da qualidade da heurística. Como usamos uma heurística simples ("disciplinas restantes"), ambas sofreram. A estratégia de profundidade (Híbrida) venceu por explorar verticalmente o espaço de estados, evitando a explosão combinatorial lateral.